# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298541456599                   -0.85    5.5         
  2   -8.300217643441       -2.78       -1.25    1.0    105ms
  3   -8.300438897016       -3.66       -1.88    2.0    118ms
  4   -8.300461787942       -4.64       -2.74    2.5    131ms
  5   -8.300464390633       -5.58       -3.20    2.6    184ms
  6   -8.300464517967       -6.90       -3.35   12.5    260ms
  7   -8.300464580700       -7.20       -3.50    2.4    131ms
  8   -8.300464615723       -7.46       -3.64    1.5    125ms
  9   -8.300464636504       -7.68       -3.83    3.1    156ms
 10   -8.300464640539       -8.39       -3.95    2.5    154ms
 11   -8.300464643790       -8.49       -4.30    1.8    175ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67460952906                   -0.70    7.5         
  2   -16.67870139338       -2.39       -1.14    1.2    270ms
  3   -16.67920321569       -3.30       -1.86    3.6    364ms
  4   -16.67927866672       -4.12       -2.75    2.8    310ms
  5   -16.67928600643       -5.13       -3.17    4.2    378ms
  6   -16.67928620086       -6.71       -3.48    2.4    288ms
  7   -16.67928621794       -7.77       -3.94    2.2    310ms
  8   -16.67928622158       -8.44       -4.59    2.1    293ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32573569199                   -0.56    6.2         
  2   -33.33281169633       -2.15       -1.00    1.8    1.01s
  3   -33.33408914540       -2.89       -1.73    4.6    1.22s
  4   -33.33425141869       -3.79       -2.61    2.1    1.05s
  5   -33.33593535732       -2.77       -2.50    5.0    1.51s
  6   -33.33694226556       -3.00       -2.51    7.8    1.34s
  7   -33.33694243458       -6.77       -2.51    2.1    977ms
  8   -33.33694173789   +   -6.16       -2.51    1.6    947ms
  9   -33.33619159189   +   -3.12       -1.95    3.0    1.46s
 10   -33.33593228891   +   -3.59       -1.87    1.6    980ms
 11   -33.33613311127       -3.70       -1.91    1.0    918ms
 12   -33.33643887916       -3.51      

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298475489777                   -0.85    5.5         
  2   -8.300266578382       -2.75       -1.59    1.0   83.3ms
  3   -8.300431411598       -3.78       -2.57    2.4    105ms
  4   -8.300383847658   +   -4.32       -2.30    7.9    202ms
  5   -8.300464054253       -4.10       -3.37    1.1   93.8ms
  6   -8.300464548559       -6.31       -3.71    3.9    154ms
  7   -8.300464637644       -7.05       -4.26    1.9    114ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32479694848                   -0.56    6.4         
  2   -33.11050221079   +   -0.67       -1.18    2.9    917ms
  3   +103.8372930604   +    2.14        0.01   15.4    3.08s
  4   -33.08268860947        2.14       -1.29    7.2    2.67s
  5   -33.24755612989       -0.78       -1.33    3.4    1.36s
  6   -33.25781370583       -1.99       -1.59    2.1    1.32s
  7   -32.72423914968   +   -0.27       -1.17    4.0    1.25s
  8   -33.33001798903       -0.22       -2.10    3.6    1.15s
  9   -33.33437359876       -2.36       -2.03    3.0    1.24s
 10   -33.33670865330       -2.63       -2.61    2.0    997ms
 11   -33.33690172863       -3.71       -2.85    2.1    958ms
 12   -33.33691737873       -4.81      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.